In [ ]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from utils.recorder_util import ModelResults
from torch import nn, optim

# enable tqdm in pandas
tqdm.pandas()

# set to True to use the gpu (if there is one available)
use_gpu = True



# select device
device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

# random seed
seed = 1234

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


recorder = ModelResults("LSTM_W2V", "Bryan Bielawa", seed)


In [ ]:
train_df = pd.read_csv('../../../trainData/trainData.csv', header=None)
train_df

In [ ]:
#borrowing Noah's Word2Vec code

def preprocess_data(text):
    lower_text = text.lower()
    preprocessed = lower_text.split()
    return preprocessed

train_df["preprocessed_text"] = (train_df["text"]).apply(preprocess_data)

In [ ]:
# Import pretrained word2vec model embeddings
#using Andrew's embeddings code

def get_embedding(text, embedding_layer, word_to_index):
    # Convert words to indices and filter out words not in the vocabulary
    indices = [word_to_index[word] for word in text if word in word_to_index]
    
    if indices:
        # Convert to a tensor and pass through the embedding layer in one go
        word_indices_tensor = torch.tensor(indices, device=device)
        vectors = embedding_layer(word_indices_tensor).cpu().detach().numpy()
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(embedding_layer.embedding_dim)

    
# Load the word2vec model

word2vec_embeddings =  np.load("../../../rawData/gloveEmbeds.npy")
word2vec_tensor = torch.tensor(word2vec_embeddings, dtype=torch.float32)
embedding_layer = nn.Embedding.from_pretrained(word2vec_tensor, freeze=True).to(device)
vocab = codecs.open("../../../rawData/vocab.txt").read().split("\n")
vocab_size = len(vocab)

word_to_index = {word: i for i, word in enumerate(vocab)}
x_train = np.array([get_embedding(text, embedding_layer, word_to_index) 
                    for text in train_df["preprocessed_text"]])
y_train = train_df["label"].values
x_train.shape, y_train.shape